In [166]:
from pypdf import PdfReader
reader = PdfReader('pdf_dataset/Civil_Law/problem_2011.pdf')

In [167]:
number_of_pages = len(reader.pages) 

In [ ]:
for i in range(10):
    print(reader.pages[i].extract_text())


In [169]:
patterns = [r'\n', r'\r', r'-\d{1,2}-', r'-\d\s\d-']

import re

def remove_annoy_from_pdf(reader, number_of_pages, patterns):
    all_text = ""  # Create an empty string variable to store all the text

    for i in range(number_of_pages):
        text = reader.pages[i].extract_text()
        # 正規表現にマッチする部分を抽出
        matches = []
        for pattern in patterns:
            matches.extend(re.findall(pattern, text))  
        text = re.sub('|'.join(patterns), '', text)
        all_text += text  # Concatenate the extracted text to the all_text variable
    
    return all_text

# Call the function and store the result in a variable
cleaned_text = remove_annoy_from_pdf(reader, number_of_pages, patterns)



In [ ]:
import re
import json

def extract_questions_from_text(text):
    # 正規表現パターンを修正して、「（配点：２）」の部分を除去
    pattern = r"〔第(\d+)問〕(.*?)(?=〔第\d+問〕|\Z)"

    # テキストから問題を抽出
    matches = re.findall(pattern, text, re.DOTALL)

    # JSON形式に整形
    json_data = {"Civil_Law": []}

    for num, content in matches:
        # 「（解答欄は ） 。， ［№\d］」と「（配点：２）」を取り除く
        cleaned_content = re.sub(r"（解答欄は ） 。， ［№\d］", "", content).strip()
        cleaned_content = re.sub(r"（配点：２）", "", cleaned_content).strip()
        json_data["Civil_Law"].append({"num": int(num), "question": cleaned_content})

    return json_data

# テキストを関数に渡して問題を抽出
json_data = extract_questions_from_text(cleaned_text)

# JSONデータを表示（整形して表示）
print(json.dumps(json_data, ensure_ascii=False, indent=2))


##### 回答を読み込み、json形式に変換する

In [5]:
# テーブルとして読み込む
import camelot

In [ ]:
tables = camelot.read_pdf('pdf_dataset/Civil_Law/answer_2011.pdf', pages='1-end')
tables

In [ ]:
import pandas as pd
df = tables[0].df
df.columns = df.iloc[0]
df = df.drop(0)
df.head(30)

In [ ]:
import pandas as pd

# データフレームの総列数を取得
total_columns = df.shape[1]

# 総列数の半分でデータフレームを分割
split_index = total_columns // 2
df1 = df.iloc[:, :split_index]
df2 = df.iloc[:, split_index:]


# 両方のデータフレームを縦に結合
df_clean = pd.concat([df1, df2], axis=0).reset_index(drop=True)

df_clean = df_clean.dropna(how='all', axis=1).reset_index(drop=True)
# 結果を表示
df_clean.head(60)


In [ ]:
for row in range(len(df_clean.iloc[:, 0])):
    if df_clean.iloc[row, 0] == "":
        df_clean.iloc[row, 0] = df_clean.iloc[row-1, 0]

df_clean.head(60)

In [ ]:
import pandas as pd
import json

# Specify the column numbers to include in the JSON
column1 = 0
column2 = 2
# Create a new dataframe with the selected columns
selected_columns = df_clean.iloc[:, [column1, column2]]

selected_columns.columns = ["num_problem", "answer"]
# Print the JSON data
selected_columns.head(60)


In [ ]:
drop_index = selected_columns[selected_columns['answer'] == ""].index
df_clean = selected_columns.drop(drop_index).reset_index(drop=True)
df_clean.head(60)

In [ ]:
import pandas as pd

df_clean['num_problem'] = df_clean['num_problem'].astype(int)

# num_problem ごとに answer を結合
grouped = df_clean.groupby('num_problem')['answer'].apply(lambda x: ', '.join(x)).reset_index()

# Converting to JSON format
json_dict = grouped.to_dict(orient='records')

# Formatting the JSON output
formatted_json = {"answer": [{"num": str(item['num_problem']), "ans": item['answer']} for item in json_dict]}

# 結果を表示

print(json.dumps(formatted_json, ensure_ascii=False, indent=2))

In [ ]:
import json

json_question = json_data["Civil_Law"]
json_answer = formatted_json["answer"]

# Create a dictionary to store the connected JSON data
connected_json = {}

# Iterate over the keys in the first JSON
for item in json_question:
    # Get the key
    key = str(item['num'])
    for item2 in json_answer:
        key2 = str(item2['num'])
        # Check if the key exists in the second JSON
        if key == key2:
            # Create a new dictionary with the connected data
            connected_json[key] = {"question": item['question'], "answer": item2['ans']}

# Convert the connected JSON dictionary to JSON format
connected_json_json = json.dumps(connected_json)

# Print the connected JSON data
print(json.dumps(connected_json, ensure_ascii=False, indent=2))

In [18]:
import re
from PyPDF2 import PdfReader
import camelot
import pandas as pd
import json

def extract_questions_from_text(path):
    # PDFファイルを読み込み
    reader = PdfReader(path)
    patterns = [r'\n', r'\r', r'-\d{1,2}-', r'-\d\s\d-']
    number_of_pages = len(reader.pages)
    all_text = ''
    for i in range(number_of_pages):
        text = reader.pages[i].extract_text()
        # 正規表現にマッチする部分を抽出
        matches = []
        for pattern in patterns:
            matches.extend(re.findall(pattern, text))  
        text = re.sub('|'.join(patterns), '', text)
        all_text += text
    # 正規表現パターンを修正して、「（配点：２）」の部分を除去
    pattern = r"〔第(\d+)問〕(.*?)(?=〔第\d+問〕|\Z)"

    # テキストから問題を抽出
    matches = re.findall(pattern, all_text, re.DOTALL)

    # JSON形式に整形
    questions_json = {"questions" : []}

    for num, content in matches:
        # 「（解答欄は ） 。， ［№\d］」と「（配点：２）」を取り除く
        cleaned_content = re.sub(r"（解答欄は ） 。， ［№\d］", "", content).strip()
        cleaned_content = re.sub(r"（配点：２）", "", cleaned_content).strip()
        questions_json["questions"].append({"num": int(num), "question": cleaned_content})

    return questions_json


def extract_answers_from_pdf(path):
    tables = camelot.read_pdf(path, pages="all")
    df = tables[0].df
    df.columns = df.iloc[0]
    df = df.drop(0)
    total_columns = df.shape[1]
    split_index = total_columns // 2
    df1 = df.iloc[:, :split_index]
    df2 = df.iloc[:, split_index:]
    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)
    df1.columns = ['Num', 'num', 'answer', 'point', 'notion']
    df2.columns = ['Num', 'num', 'answer', 'point', 'notion']
    df_clean = pd.concat([df1, df2], axis=0).reset_index(drop=True)
    df_clean = df_clean.dropna(axis=1, how="all").reset_index(drop=True)
    for row in range(len(df_clean.iloc[:, 0])):
        if df_clean.iloc[row, 0] == "":
            df_clean.iloc[row, 0] = df_clean.iloc[row-1, 0]

    # Specify the column numbers to include in the JSON
    column1 = 0
    column2 = 2
    # Create a new dataframe with the selected columns
    selected_columns = df_clean.iloc[:, [column1, column2]]

    selected_columns.columns = ["num_problem", "answer"]
    
    drop_index = selected_columns[selected_columns['answer'] == ""].index
    df_clean = selected_columns.drop(drop_index).reset_index(drop=True)

    df_clean['num_problem'] = df_clean['num_problem'].astype(int)

    # num_problem ごとに answer を結合
    grouped = df_clean.groupby('num_problem')['answer'].apply(lambda x: ', '.join(x)).reset_index()

    # Converting to JSON format
    json_dict = grouped.to_dict(orient='records')

    # Formatting the JSON output
    answers_json = {"answers": [{"num": str(item['num_problem']), "ans": item['answer']} for item in json_dict]}

    return answers_json

def connect_question_and_answer(question_json, answer_json):
    questions = question_json['questions']
    answers = answer_json['answers']
    connected_json = []
    for question in questions:
        for answer in answers:
            if str(question['num']) == str(answer['num']):
                connected_json.append({"question": question['question'], "answer": answer['ans']})

    return connected_json

def process_pairs(start_year, end_year):
    legal_questions_and_answer_dataset = {}
    for year in range(start_year, end_year+1):
        question_file = f"pdf_dataset/Civil_Law/Preliminary_exam/question_{year}.pdf"
        answer_file = f"pdf_dataset/Civil_Law/Preliminary_exam/answer_{year}.pdf"
        answer_json = extract_answers_from_pdf(answer_file)
        question_json = extract_questions_from_text(question_file)
        legal_questions_and_answer_dataset[str(year)] = connect_question_and_answer(question_json, answer_json)
    
    return legal_questions_and_answer_dataset

In [20]:
legal_questions_and_answer_dataset = process_pairs(2011, 2022)

/opt/anaconda3/envs/NER/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniJIS-UTF16-H not implemented yet
  warnings.warn(


In [21]:
legal_questions_and_answer_dataset

{'2011': [{'question': '制限行為能力者に関する次のアからオまでの各記述のうち，誤っているものを組み合わせたものは，後記１から５までのうちどれか ア．未成年者は，単に義務を免れる法律行為について，その法定代理人の同意を得ないですることができる。イ．未成年者又は成年被後見人を相手方として意思表示をした者は，法定代理人がその意思表示を知る前は，その未成年者又は成年被後見人に対してその意思表示に係る法律効果を主張することができない。ウ．代理人が保佐開始の審判を受けたときは，代理権は消滅する。エ．成年被後見人は，行為能力者であることを信じさせるため詐術を用いたときであっても，その行為を取り消すことができる。オ．未成年の子が婚姻をするには，原則として父母の同意を得なければならないが，成年被後見人が婚姻をするには，その成年後見人の同意を要しない。１．ア イ ２．ア オ ３．イ ウ ４．ウ エ ５．エ オ',
   'answer': '4'},
  {'question': '無権代理に関する次のアからオまでの各記述のうち，判例の趣旨に照らし正しいものを組み合わせたものは，後記１から５までのうちどれか ア．本人が無権代理人に対して無権代理行為を追認した場合でも，相手方は，その事実を知らなければ取消権を行使することができる。イ．無権代理行為の相手方は，本人に対して相当の期間を定めて，その期間内に追認するか否かを催告することができ，本人がその期間内に確答をしないときは，追認したものとみなされる。ウ．無権代理行為の相手方は，表見代理の主張をしないで，無権代理人に対し履行又は損害賠償の請求をすることができるが，これに対し無権代理人は，表見代理の成立を主張してその責任を免れることができる。エ．無権代理人が本人を代理して第三者の貸金債務につき本人名義で連帯保証契約を締結した後 本人が追認も追認拒絶もしないまま死亡し 無権代理人が他の者と共に本人を相続した場，，合 他の共同相続人全員の追認がなくても 無権代理人が本人から相続により承継した部分に，，ついて，無権代理行為は有効となる。オ．無権代理人が本人所有の土地に抵当権を設定したため，本人が抵当権設定登記の抹消登記請求訴訟を提起した後死亡し，無権代理人が本人を相続したとしても，無権代理行為は，有効とならない。１．ア エ 

In [30]:
import os
import json

# JSONファイルの保存先パス
output_file = "json_data/Civil_law/Preliminary_examlegal_dataset_2011_to_2022.json"

# ディレクトリが存在しない場合は作成
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# legal_questions_and_answer_datasetをJSON形式で保存
with open(output_file, "w") as f:
    json.dump(legal_questions_and_answer_dataset, f, ensure_ascii=False, indent=2)

## 司法試験の問題の前処理

In [102]:
import re
from PyPDF2 import PdfReader
import camelot
import pandas as pd
import json
import itertools

def extract_questions_from_text(path):
    # PDFファイルを読み込み
    reader = PdfReader(path)
    patterns = [r'\n', r'\r', r'-\d{1,2}-', r'-\d\s\d-']
    number_of_pages = len(reader.pages)
    all_text = ''
    for i in range(number_of_pages):
        text = reader.pages[i].extract_text()
        # 正規表現にマッチする部分を抽出
        matches = []
        for pattern in patterns:
            matches.extend(re.findall(pattern, text))  
        text = re.sub('|'.join(patterns), '', text)
        all_text += text
    # 正規表現パターンを修正して、「（配点：２）」の部分を除去
    pattern = r"〔第(\d+)問〕(.*?)(?=〔第\d+問〕|\Z)"

    # テキストから問題を抽出
    matches = re.findall(pattern, all_text, re.DOTALL)

    # JSON形式に整形
    questions_json = {"questions" : []}

    for num, content in matches:
        cleaned_content = re.sub(r'（配点：\d）', "", content).strip()
        cleaned_content = re.sub(r'\[№\d+\]', "", cleaned_content).strip()
        cleaned_content = re.sub(r'№\d+', "", cleaned_content).strip()
        cleaned_content = re.sub(r'解答欄は', "", cleaned_content).strip()
        cleaned_content = re.sub(r'順不同', "", cleaned_content).strip()
        cleaned_content = re.sub(r'[。、）（,，\[\]［］]', "", cleaned_content).strip()
        questions_json["questions"].append({"num": int(num), "question": cleaned_content})


    return questions_json

def extract_answers_from_pdf(path):
    tables = camelot.read_pdf(path, pages="all")
    df = tables[0].df
    df.columns = df.iloc[0]
    df = df.drop(0)
    total_columns = df.shape[1]

    column_nums = total_columns // 5
    split_index = total_columns // column_nums

    # 列数に応じてDataFrameを分割する処理を一般化
    dfs = [df.iloc[:, i*split_index:(i+1)*split_index].reset_index(drop=True) for i in range(column_nums)]
    for df_part in dfs:
        df_part.columns = ['Num', 'num', 'answer', 'point', 'notion']

    df_clean = pd.concat(dfs, axis=0).reset_index(drop=True)
    for row in range(len(df_clean.iloc[:, 0])):
        if df_clean.iloc[row, 0] == "":
            df_clean.iloc[row, 0] = df_clean.iloc[row-1, 0]

    # Specify the column numbers to include in the JSON
    column1 = 0
    column2 = 2
    column3 = 4
    # Create a new dataframe with the selected columns
    selected_columns = df_clean.iloc[:, [column1, column2, column3]]

    selected_columns.columns = ["num_problem", "answer", "Notion"]

    drop_index = selected_columns[selected_columns['answer'] == ""].index
    df_clean = selected_columns.drop(drop_index).reset_index(drop=True)

    df_clean['num_problem'] = df_clean['num_problem'].astype(int)

    # num_problem ごとに answer を結合し、Notion 列が '順'、'不'、'同' をそれぞれ含むかどうかをチェック
    grouped = df_clean.groupby('num_problem').agg({
        'answer': lambda x: ', '.join(x),
        'Notion': lambda x: any(
            ('順' in notion) and ('不' in notion) and ('同' in notion) for notion in x
        )
    }).reset_index()

    # Converting to JSON format
    json_dict = grouped.to_dict(orient='records')

    # Formatting the JSON output
    answers_json = {
        "answers": [
            {"num": str(item['num_problem']), "ans": item['answer'], "any_order": item['Notion']}
            for item in json_dict
        ]
    }
    return answers_json

def process_answers(answers_json):
    processed_answers = []

    for answer in answers_json["answers"]:
        num = answer['num']
        ans_text = answer['ans']
        any_order = answer['any_order']

        if "\n" in ans_text:
            ans_text = ans_text.replace("\n", ", ")

        if ans_text.isdigit() and len(ans_text) > 1:
            ans_text = ', '.join(list(ans_text))
      
        # 順不同の場合、全ての順列を生成
        if any_order:
            # カンマとスペースで分割して順列を生成
            ans_parts = ans_text.split(', ')
            permutations = itertools.permutations(ans_parts)
            all_combinations = [', '.join(permutation) for permutation in permutations]
        else:
            all_combinations = [ans_text]  # 順不同でない場合は元の回答を使用

        processed_answers.append({"num": num, "ans": all_combinations})
        answers_json_2 = {"answers": processed_answers}
    return answers_json_2

def connect_question_and_answer(question_json, answers_json_2):
    questions = question_json['questions']
    answers = answers_json_2['answers']
    connected_json = []

    for question in questions:
        for answer in answers:
            if str(question['num']) == answer['num']:
                # 回答リストの処理
                answer_texts = answer['ans']
                connected_json.append({"question_text": question['question'], "answers": answer_texts})

    return connected_json


def process_pairs(start_year, end_year, path):
    legal_questions_and_answer_dataset = {}
    for year in range(start_year, end_year+1):
        question_file = f"{path}/question_{year}.pdf"
        answer_file = f"{path}/answer_{year}.pdf"
        question_json = extract_questions_from_text(question_file)
        answer_json = extract_answers_from_pdf(answer_file)
        answer_json_2 = process_answers(answer_json)
        question_answer_pairs = connect_question_and_answer(question_json, answer_json_2)

        # 各質問に対して記述と選択肢を抽出
        for pair in question_answer_pairs:
            new_text = pair["question_text"]

            # 各記述を抽出する正規表現
            new_text = re.sub(r"\s+", "", new_text)
            statements_pattern = r"(\d[．\.].*?)(?=\d[．\.]|$)"

            # 各記述の抽出
            statements = re.sub(statements_pattern, "", new_text, re.DOTALL)
            choices = re.findall(statements_pattern, new_text, re.DOTALL)
           
            statements = statements.strip() if statements.strip() else "None"
            choices = choices if choices else ["None"]

            # 結果をpairに追加
            pair["question_body"] = statements
            pair["options"] = choices
        
        legal_questions_and_answer_dataset[str(year)] = question_answer_pairs
    
    return legal_questions_and_answer_dataset


In [75]:
Civil_Pre_pdf = "pdf_dataset/Civil_Law/Preliminary_exam"
Civil_Pre_json = "json_data/Civil_law/Preliminary_exam"
Civil_Bar_pdf = "pdf_dataset/Civil_Law/Bar_exam"
Civil_Bar_json = "json_data/Civil_law/Bar_exam"

Cons_Pre_pdf = "pdf_dataset/Constitution_law/Preliminary_exam"
Cons_Pre_json = "json_data/Constitution_law/Preliminary_exam"
Cons_Bar_pdf = "pdf_dataset/Constitution_law/Bar_exam"
Cons_Bar_json = "json_data/Constitution_law/Bar_exam"

Crim_Pre_pdf = "pdf_dataset/Criminal_law/Preliminary_exam"
Crim_Pre_json = "json_data/Criminal_law/Preliminary_exam"
Crim_Bar_pdf = "pdf_dataset/Criminal_law/Bar_exam"
Crim_Bar_json = "json_data/Criminal_law/Bar_exam"

In [103]:
civil_pre = process_pairs(2011, 2022, Civil_Pre_pdf)
civil_bar = process_pairs(2008, 2022, Civil_Bar_pdf)
cons_pre = process_pairs(2011, 2022, Cons_Pre_pdf)
cons_bar = process_pairs(2008, 2022, Cons_Bar_pdf)
crim_pre = process_pairs(2011, 2022, Crim_Pre_pdf)
crim_bar = process_pairs(2008, 2022, Crim_Bar_pdf)

/opt/anaconda3/envs/NER/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniJIS-UTF16-H not implemented yet
  warnings.warn(
/opt/anaconda3/envs/NER/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniJIS-UTF16-H not implemented yet
  warnings.warn(
/opt/anaconda3/envs/NER/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniJIS-UTF16-H not implemented yet
  warnings.warn(


In [ ]:
import os
import json

# JSONファイルの保存先パス
[output_file1, output_file2, output_file3, output_file4, output_file5, output_file6] = [
    "json_data/Civil_law/Preliminary_examlegal_dataset_2011_to_2022_v6.json",
    "json_data/Civil_law/Bar_examlegal_dataset_2008_to_2022_v6.json",
    "json_data/Constitution_law/Preliminary_examlegal_dataset_2011_to_2022_v6.json",
    "json_data/Constitution_law/Bar_examlegal_dataset_2008_to_2022_v6.json",
    "json_data/Criminal_law/Preliminary_examlegal_dataset_2011_to_2022_v6.json",
    "json_data/Criminal_law/Bar_examlegal_dataset_2008_to_2022_v6.json"
]

# ディレクトリが存在しない場合は作成
os.makedirs(os.path.dirname(output_file1), exist_ok=True)
os.makedirs(os.path.dirname(output_file2), exist_ok=True)
os.makedirs(os.path.dirname(output_file3), exist_ok=True)
os.makedirs(os.path.dirname(output_file4), exist_ok=True)
os.makedirs(os.path.dirname(output_file5), exist_ok=True)
os.makedirs(os.path.dirname(output_file6), exist_ok=True)

# legal_questions_and_answer_datasetをJSON形式で保存
with open(output_file1, "w") as f:
    json.dump(civil_pre, f, ensure_ascii=False, indent=2)
with open(output_file2, "w") as f:
    json.dump(civil_bar, f, ensure_ascii=False, indent=2)
with open(output_file3, "w") as f:
    json.dump(cons_pre, f, ensure_ascii=False, indent=2)
with open(output_file4, "w") as f:
    json.dump(cons_bar, f, ensure_ascii=False, indent=2)
with open(output_file5, "w") as f:
    json.dump(crim_pre, f, ensure_ascii=False, indent=2)
with open(output_file6, "w") as f:
    json.dump(crim_bar, f, ensure_ascii=False, indent=2)


In [10]:
import json

output_files = [
    "json_data/Civil_law/Preliminary_exam/Preliminary_examlegal_dataset_2011_to_2022_v6.json",
    "json_data/Civil_law/Bar_exam/Bar_examlegal_dataset_2008_to_2022_v6.json",
    "json_data/Constitution_law/Preliminary_examlegal_dataset_2011_to_2022_v6.json",
    "json_data/Constitution_law/Bar_examlegal_dataset_2008_to_2022_v6.json",
    "json_data/Criminal_law/Preliminary_examlegal_dataset_2011_to_2022_v6.json",
    "json_data/Criminal_law/Bar_examlegal_dataset_2008_to_2022_v6.json"
]

concatenated_data = []

for file in output_files:
    with open(file, "r") as f:
        component = json.load(f)
        for key in component.keys():
            data = component[key]
            concatenated_data.extend(data)

concatenated_data

[{'question_text': '制限行為能力者に関する次のアからオまでの各記述のうち誤っているものを組み合わせたものは後記１から５までのうちどれか    ア．未成年者は単に義務を免れる法律行為についてその法定代理人の同意を得ないですることができるイ．未成年者又は成年被後見人を相手方として意思表示をした者は法定代理人がその意思表示を知る前はその未成年者又は成年被後見人に対してその意思表示に係る法律効果を主張することができないウ．代理人が保佐開始の審判を受けたときは代理権は消滅するエ．成年被後見人は行為能力者であることを信じさせるため詐術を用いたときであってもその行為を取り消すことができるオ．未成年の子が婚姻をするには原則として父母の同意を得なければならないが成年被後見人が婚姻をするにはその成年後見人の同意を要しない１．ア イ ２．ア オ ３．イ ウ ４．ウ エ ５．エ オ',
  'answers': ['4'],
  'question_body': '制限行為能力者に関する次のアからオまでの各記述のうち誤っているものを組み合わせたものは後記１から５までのうちどれかア．未成年者は単に義務を免れる法律行為についてその法定代理人の同意を得ないですることができるイ．未成年者又は成年被後見人を相手方として意思表示をした者は法定代理人がその意思表示を知る前はその未成年者又は成年被後見人に対してその意思表示に係る法律効果を主張することができないウ．代理人が保佐開始の審判を受けたときは代理権は消滅するエ．成年被後見人は行為能力者であることを信じさせるため詐術を用いたときであってもその行為を取り消すことができるオ．未成年の子が婚姻をするには原則として父母の同意を得なければならないが成年被後見人が婚姻をするにはその成年後見人の同意を要しない',
  'options': ['１．アイ', '２．アオ', '３．イウ', '４．ウエ', '５．エオ']},
 {'question_text': '無権代理に関する次のアからオまでの各記述のうち判例の趣旨に照らし正しいものを組み合わせたものは後記１から５までのうちどれか    ア．本人が無権代理人に対して無権代理行為を追認した場合でも相手方はその事実を知らなければ取消権を行使することができるイ．無権代理行為の相手方は本人に対して相

In [ ]:
# Save the concatenated data to a new JSON file
output_file = "json_data/concatenated_data_v6.json"
with open(output_file, "w") as f:
    json.dump(concatenated_data, f, ensure_ascii=False, indent=2)

## jsonを加工する

In [1]:
import json
with open("json_data/Civil_law/Preliminary_exam/Preliminary_examlegal_dataset_2011_to_2022.json","r") as f:
    data = json.load(f)

In [98]:
data = extract_answers_from_pdf("pdf_dataset/Civil_Law/Preliminary_exam/answer_2011.pdf")
process_answers(data)


TypeError: string indices must be integers

In [106]:
nums = "4, 6, 7"
lists = nums.split(", ")

In [108]:
permutations = itertools.permutations(lists)
all_combinations = [' , '.join(permutation) for permutation in permutations]

In [109]:
all_combinations

['4 , 6 , 7', '4 , 7 , 6', '6 , 4 , 7', '6 , 7 , 4', '7 , 4 , 6', '7 , 6 , 4']